In [19]:
import torch
from torchvision import datasets, transforms
import plotly.graph_objects as go
import plotly.express as px
import torch.nn as nn
import numpy as np

Download MNIST

In [26]:
datasets.MNIST.resources = [
            ('https://ossci-datasets.s3.amazonaws.com/mnist/train-images-idx3-ubyte.gz', 'f68b3c2dcbeaaa9fbdd348bbdeb94873'),
            ('https://ossci-datasets.s3.amazonaws.com/mnist/train-labels-idx1-ubyte.gz', 'd53e105ee54ea40749a09fcbcd1e9432'),
            ('https://ossci-datasets.s3.amazonaws.com/mnist/t10k-images-idx3-ubyte.gz', '9fb629c4189551a2d022fa330f9573f3'),
            ('https://ossci-datasets.s3.amazonaws.com/mnist/t10k-labels-idx1-ubyte.gz', 'ec29112dd5afa0611ce80d1b7f02629c')
        ]

transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)),
        transforms.Lambda(lambda x: x.view(-1))
        ])
training_data = datasets.MNIST('./data', train=True, download=True,transform=transform)
testing_data = datasets.MNIST('./data', train=False,transform=transform)



Create a dataloader

In [27]:
train_loader = torch.utils.data.DataLoader(dataset=training_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=testing_data, batch_size=64, shuffle=True)

Look at images

In [28]:
img_rgb = np.array([[[255, 0, 0], [0, 255, 0], [0, 0, 255]],
                    [[0, 255, 0], [0, 0, 255], [255, 0, 0]]
                   ], dtype=np.uint8)
img = 255 - train_loader.dataset.data[0]
fig = px.imshow(img)
layout = px.imshow(img, color_continuous_scale='gray').layout
fig.layout.coloraxis = layout.coloraxis
fig.show()

Create a neural network

In [37]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 100)
        self.fc2 = nn.Linear(100, 10)
        self.Relu = nn.ReLU()

    def forward(self, x):
        x = self.Relu(self.fc1(x))
        x = self.fc2(x)
        
        return x
net = NeuralNet()

Create optimizer and loss function

In [38]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.001)
loss_metric = nn.CrossEntropyLoss()

In [39]:
accuracies = []
losses = []
for epoch in range(10):
    correct = 0
    train_loss = 0
    for i, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = net.forward(x)
        loss = loss_metric(outputs, y)
        train_loss += loss
        pred = outputs.data.max(1)[1]  # get the index of the max log-probability
        correct_array = pred.eq(y.data).cpu()
        correct += correct_array.sum()
        loss.backward()
        optimizer.step()

    accuracy = correct/len(train_loader.dataset)
    print('Epoch:', epoch+1, 'Loss:', loss, 'Accuracy:', accuracy)
    losses.append(loss.detach().numpy())
    accuracies.append(accuracy.numpy())
    epoch += 1

Epoch: 1 Loss: tensor(1.0747, grad_fn=<NllLossBackward>) Accuracy: tensor(0.5945)
Epoch: 2 Loss: tensor(0.7645, grad_fn=<NllLossBackward>) Accuracy: tensor(0.8308)
Epoch: 3 Loss: tensor(0.6137, grad_fn=<NllLossBackward>) Accuracy: tensor(0.8612)
Epoch: 4 Loss: tensor(0.5169, grad_fn=<NllLossBackward>) Accuracy: tensor(0.8746)
Epoch: 5 Loss: tensor(0.5220, grad_fn=<NllLossBackward>) Accuracy: tensor(0.8821)
Epoch: 6 Loss: tensor(0.3779, grad_fn=<NllLossBackward>) Accuracy: tensor(0.8887)
Epoch: 7 Loss: tensor(0.2752, grad_fn=<NllLossBackward>) Accuracy: tensor(0.8934)
Epoch: 8 Loss: tensor(0.4185, grad_fn=<NllLossBackward>) Accuracy: tensor(0.8970)
Epoch: 9 Loss: tensor(0.3966, grad_fn=<NllLossBackward>) Accuracy: tensor(0.9004)
Epoch: 10 Loss: tensor(0.2908, grad_fn=<NllLossBackward>) Accuracy: tensor(0.9035)


Look at the learning curves

In [40]:
fig = go.Figure()
fig = go.Figure(data=go.Scatter(x=list(range(len(accuracies))), y=accuracies))
fig.update_layout(title='Accuracy vs epoch',
                   xaxis_title='Epoch',
                   yaxis_title='Accuracy')
fig.show()

In [41]:
fig = go.Figure()
fig = go.Figure(data=go.Scatter(x=list(range(len(losses))), y=losses))
fig.update_layout(title='Loss vs epoch',
                   xaxis_title='Epoch',
                   yaxis_title='Loss')
fig.show()

Get test accuracy

In [43]:
net.eval()
total = 0
correct = 0
for i, (x, y) in enumerate(test_loader):
    output = net.forward(x)
    pred = output.data.max(1)[1]  # get the index of the max log-probability
    correct_array = pred.eq(y.data).cpu()
    correct += correct_array.sum()
    total += len(correct_array)

testing_acc = 100. * correct / total
testing_acc

tensor(90.9900)

# Things to test
- Change the batch size (16, 32, 64, 128)
- Change the epochs (5, 10, 20, 50)
- Change the learning rate (0.1, 0.01 0.001, 0.0001)
- Change the network depth (add more layers)
- Change the layer size (50, 100, 200, 300)

For each of the above, create a plot and write an observation how your test accuracy, learning curve changes with it. 
Show in a ppt format. 
